In [69]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix

In [70]:
#Convert MNIST Image files into a Tensor of 4-Dimensions(# of images,Height, Width, Colour channels) 
transform=transforms.ToTensor()

In [71]:
train_data= datasets.MNIST(root='/cnn_data',train=True,download=True,transform=transform)

In [72]:
test_data= datasets.MNIST(root='/cnn_data',train=False,download=True,transform=transform)

In [73]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [74]:
train_loader= DataLoader(train_data,batch_size=10,shuffle=True)
test_loader= DataLoader(test_data, batch_size=10, shuffle=False)

In [75]:
#Define our CNN mdel
#Describe convolutional layer and what its doing(2 convolutional layers)
conv1=nn.Conv2d(1,6,3,1)
conv2=nn.Conv2d(6,16,3,1)

In [76]:
#Grab 1 MNIST  record/image
for i , (X_Train,y_train) in enumerate(train_data):
    break

In [77]:
X_Train.shape

torch.Size([1, 28, 28])

In [78]:
x=X_Train.view(1,1,28,28)

In [79]:
#Perform our first convolution
x=F.relu(conv1(x))

In [80]:
x.shape

torch.Size([1, 6, 26, 26])

In [81]:
x=F.max_pool2d(x,2,2)

In [82]:
#Do our second convolutional layer
x=F.relu(conv2(x))


In [83]:
x.shape

torch.Size([1, 16, 11, 11])

In [84]:
x=F.max_pool2d(x,2,2)

In [85]:
#Model class
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,6,3,1)
        self.conv2=nn.Conv2d(6,16,3,1)
        #fully connected layer
        self.fc1=nn.Linear(5*5*16,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
        
    def forward(self,X):
        X=F.relu(self.conv1(X))
        X=F.max_pool2d(X,2,2)
        X=F.relu(self.conv2(X))
        X=F.max_pool2d(X,2,2)  
        
        X=X.view(-1,16*5*5)
        X=F.relu(self.fc1(X))
        X=F.relu(self.fc2(X))
        X=self.fc3(X)
        return F.log_softmax(X, dim=1)

        
    

In [86]:
#create an instance of our model
torch.manual_seed(41)
model=ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [87]:
#Loss function Optimizer
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [88]:
import time
start_time=time
#create variables to track things
epochs=5
train_losses=[]
test_losses=[]
train_correct=[]
test_correct=[]

for i in range(epochs):
    trn_corr=0
    tst_corr=0
    
    for b,(X_train,y_train) in enumerate(train_loader):
        b+=1
        y_pred=model(X_train)
        loss=criterion(y_pred,y_train)
        
        predicted=torch.max(y_pred.data,1)[1]
        batch_corr=(predicted==y_train).sum()
        trn_corr+=batch_corr
        
        #Update our parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #print out some results
        if b%600==0:
            print(f'Epoch:{i} Batch: {b} loss:{loss.item()}')
            
    train_losses.append(loss)
    train_correct.append(trn_corr)
    
    with torch.no_grad():
        for b,(X_test,y_test) in enumerate(test_loader):
            y_val=model(X_test)
            predicted=torch.max(y_val.data,1)[1]
            tst_corr += (predicted==y_test).sum()
            
    loss=criterion(y_val,y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)    
            
            
current_time=time.time()
total=current_time-start_time
print(f'Training Took: {total/60} minutes')

Epoch:0 Batch: 600 loss:0.16236117482185364
Epoch:0 Batch: 1200 loss:0.161472886800766
Epoch:0 Batch: 1800 loss:0.4654846787452698
Epoch:0 Batch: 2400 loss:0.13513222336769104
Epoch:0 Batch: 3000 loss:0.007508326321840286
Epoch:0 Batch: 3600 loss:0.3583109378814697
Epoch:0 Batch: 4200 loss:0.0038872763980180025
Epoch:0 Batch: 4800 loss:0.002853335114195943
Epoch:0 Batch: 5400 loss:0.031907182186841965
Epoch:0 Batch: 6000 loss:0.0003448243369348347
Epoch:1 Batch: 600 loss:0.004038522019982338
Epoch:1 Batch: 1200 loss:0.19499465823173523
Epoch:1 Batch: 1800 loss:0.004099772311747074
Epoch:1 Batch: 2400 loss:0.00392061285674572
Epoch:1 Batch: 3000 loss:0.03267955407500267
Epoch:1 Batch: 3600 loss:0.24789121747016907
Epoch:1 Batch: 4200 loss:0.028354685753583908
Epoch:1 Batch: 4800 loss:0.0003152892750222236
Epoch:1 Batch: 5400 loss:0.0015493209939450026
Epoch:1 Batch: 6000 loss:0.30821070075035095
Epoch:2 Batch: 600 loss:0.06283091008663177
Epoch:2 Batch: 1200 loss:0.0017618127167224884
E

TypeError: unsupported operand type(s) for -: 'float' and 'module'

In [ ]:
train_losses=[tl.item() for tl in train_losses]
#graph the loss at epoch
plt.plot(train_losses,label="Training Loss")
plt.plot(test_losses,label="Validation Loss")
plt.title("Loss at epoch")
plt.legend()

In [ ]:
#graph the accuracy at each end of the epoch
plt.plot([t/600 for t in train_correct],label="Training Accuracy")
plt.plot([t/100 for t in test_correct],label="Validation Accuracy")
plt.title("Accuracy at the end of each Epoch")
plt.legend()



In [ ]:
test_load_everything=DataLoader(test_data,,batch_size=10000,shuffle=False)


In [ ]:
with torch.no_grad():
    correct=0
    for X_test, y_test in test_load_everything:
        y_val=model(X_test)
        predicted=torc.max(y_val,1)[1]
        correct+=(predicted == y_test).sum()
        

In [ ]:
correct.item()/len(test_data).sum()

In [ ]:
test_data(4143)

In [ ]:
#Grab just the data
test_data[4143][0]


In [ ]:
#Reshape it
test_data[4143][0].reshape(28,28)

In [ ]:
plt.imshow(test_data[4143][0].reshape(28,28)

In [ ]:
model.eval()
with torch.no_grad():
    new_prediction=model(test_data[4143][0].view(1,1,28,28))

In [ ]:
new_prediction

In [ ]:
new_prediction.argmax()